In [1]:
import numpy as np
n_features = 'ascvd'
term_pred = 'long_term'
data_file_name = 'data_after_y10_origin_at_10_ascvd'
training_id_file_name = 'all_training_ID_outerloop_cohort_10_26'
endpt = 16;
eval_times = 365.25*np.r_[np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,event,time,ascvd,sex,race,age,cholesterol,hdl,sbp,hbp.medication,smoker,diabetes
0,100016012504,0,5258.5,0.015689,0,0,27,186.0,38.0,134.000000,0,1,0
1,100023004268,0,5560.5,0.003673,0,0,35,166.0,43.0,112.000000,0,0,0
2,100033323702,0,5381.5,0.001691,0,0,28,234.0,52.0,100.000000,0,0,0
3,100056526386,0,5511.5,0.022868,0,1,34,199.0,44.0,132.000000,0,0,0
4,100061300991,0,5403.5,0.017240,0,1,26,200.0,40.0,118.000000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,0.004976,1,0,35,225.0,57.0,120.000000,0,0,0
4196,416771521620,0,5586.5,0.008172,0,1,28,204.0,76.0,106.000000,0,0,0
4197,416783315386,0,5563.5,0.001775,1,0,32,159.0,58.0,120.000000,0,0,0
4198,416796224310,0,5452.5,0.000524,1,1,32,164.0,55.0,110.843678,0,0,0


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,7,8,9
0,100016012504,0,5258.5,0.328559,-1.120553,-0.959853,-0.814396,0.239613,-1.105713,1.858692,-0.123404,1.596965,-0.14629
1,100023004268,0,5560.5,-0.169216,-1.120553,-0.959853,1.388979,-0.356702,-0.746751,0.269586,-0.123404,-0.626188,-0.14629
2,100033323702,0,5381.5,-0.251336,-1.120553,-0.959853,-0.538974,1.670768,-0.100620,-0.597199,-0.123404,-0.626188,-0.14629
3,100056526386,0,5511.5,0.625972,-1.120553,1.041826,1.113557,0.627217,-0.674959,1.714228,-0.123404,-0.626188,-0.14629
4,100061300991,0,5403.5,0.392802,-1.120553,1.041826,-1.089818,0.657033,-0.962128,0.702979,-0.123404,1.596965,-0.14629
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,-0.115260,0.892416,-0.959853,1.388979,1.402426,0.258341,0.847443,-0.123404,-0.626188,-0.14629
4196,416771521620,0,5586.5,0.017143,-1.120553,1.041826,-0.538974,0.776296,1.622396,-0.163807,-0.123404,-0.626188,-0.14629
4197,416783315386,0,5563.5,-0.247862,0.892416,-0.959853,0.562713,-0.565412,0.330134,0.847443,-0.123404,-0.626188,-0.14629
4198,416796224310,0,5452.5,-0.299701,0.892416,1.041826,0.562713,-0.416334,0.114757,0.186062,-0.123404,-0.626188,-0.14629


In [9]:
# Outerloop:
for fold in range(25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir,'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 0:

Using nesterov accelerated gradient
training model
cost: 1.074746, iteration: 0
cost: 1.063336, iteration: 46
cost: 1.056660, iteration: 69
cost: 1.050033, iteration: 92
cost: 1.043464, iteration: 115
cost: 1.036956, iteration: 138
cost: 1.030508, iteration: 161
cost: 1.024118, iteration: 184
cost: 1.017788, iteration: 207
cost: 1.011515, iteration: 230
cost: 1.005300, iteration: 253
cost: 0.999142, iteration: 276
cost: 0.993041, iteration: 299
cost: 0.986996, iteration: 322
cost: 0.981006, iteration: 345
cost: 0.975071, iteration: 368
cost: 0.969190, iteration: 391
cost: 0.963364, iteration: 414
cost: 0.957591, iteration: 437
cost: 0.951870, iteration: 460
cost: 0.946203, iteration: 483
cost: 0.940587, iteration: 506
cost: 0.935023, iteration: 529
cost: 0.929510, iteration: 552
cost: 0.924048, iteration: 575
cost: 0.918636, iteration: 598
cost: 0.913273, iteration: 621
cost: 0.907960, iteration: 644
cost: 0.902695, iteration: 667
cost: 0.897479, iteration: 690
cost: 0.892311